### 1. Level shift anomaly detection

Goal: To detect sudden drops or increases in data values, enabling us to provide timely alerts.

[ADTK](https://adtk.readthedocs.io/en/stable/)

### 2. Isolation forest anomaly detection
Help us monitor collections of abnormal data points. In our use case, they are used to issue warnings when the value have not returned to normal.

[sklearn.ensemble.IsolationForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html)


given thrive usually works with software and mobile data, i'll use app's daily active users (DAU) to describe the data issues that may arise in real-world scenarios. I'll do an anomaly detection system to perform backtesting, validating the feasibility of this algorithm and demonstrating how it can be applied in situations.

In [ ]:
1+3

4

## Load data

In [ ]:
import pandas as pd
import plotly.graph_objects as go

def load_data(filename,x_column,y_column):
    df = pd.read_csv(url)
    df.index = pd.to_datetime(df[x_column])
    df.drop(columns=x_column,inplace=True)
    df[y_column] = df[y_column].astype(float)
    return df

def plot_base_line(df,y_column):
    figure = go.Figure()
    figure.add_trace(go.Scatter(name=y_column, x = df.index, y=df[y_column], marker=dict(color='rgba(50,50,50,0.3)')))
    figure.show()

In [ ]:
url = 'https://raw.githubusercontent.com/amenti4k/webchatter/main/trialthriveportcodau.csv?token=GHSAT0AAAAAACQ5GWW4VE4JHPCCFCK5GAGSZQ7LZWQ'
filename = 'url'
x_column = 'Date'
y_column = 'DAU'
df = load_data(filename,x_column,y_column)

# df = df['2022-09-01':'2022-11-12']
plot_base_line(df,y_column)

## Level shift anomaly detection module

In [ ]:
!pip install adtk

In [ ]:
from adtk.data import validate_series
from adtk.detector import LevelShiftAD
import pandas as pd
import plotly.graph_objects as go
import warnings

In [ ]:
from adtk.data import validate_series
from adtk.detector import LevelShiftAD
import warnings
warnings.simplefilter(action="ignore", category=UserWarning)


def level_shift_anomaly(series,config={'c':1,'side':'both','window':3}):
    s = validate_series(series)
    model = LevelShiftAD(c=config['c'], side=config['side'],window = config['window'])
    anomalies = model.fit_detect(s)
    return anomalies

def _join_df_with_anomaly(df, anomalies, anomaly_column):
    anomalies = pd.DataFrame(anomalies)
    anomalies.fillna(False, inplace=True)
    anomalies.reset_index(drop=True)
    df[anomaly_column]=anomalies.to_numpy()
    return df

def plot_anomalies(df, y_column, config):
    figure = go.Figure()
    # plot baseline
    figure.add_trace(go.Scatter(name=y_column, x = df.index, y=df[y_column], marker=dict(color='rgba(50,50,50,0.3)')))

    # plot anomaly points
    anomaly_df = df
    anomaly_df = anomaly_df[anomaly_df[config['anomaly_column']]==True]

    figure.add_trace(go.Scatter(name=config['legend_name'], x = anomaly_df.index, y=anomaly_df[y_column],
        mode='markers',
        marker=dict(color=config['color'],size=10)))

    figure.update_layout(
            title= 'DAU (simulated)',
            xaxis_title='date',
            yaxis_title='DAU',
            legend_title="Anomaly Type",
        )

    figure.show()

In [ ]:
config={
    'anomaly_column':'levelshift_ad',
    'legend_name': 'levelshift anomaly',
    'color':'rgba(249,123,34,0.8)'
}

anomalies = level_shift_anomaly(df)
df_anomalies = _join_df_with_anomaly(df,anomalies,config['anomaly_column'])
plot_anomalies(df_anomalies,y_column,config)

## Isolation forest anomaly detection module

In [ ]:
from sklearn.ensemble import IsolationForest

def isolation_forest(df):
    df_without_index = df.reset_index(drop=True)
    model = IsolationForest(bootstrap=True,contamination=0.1, max_samples=0.2)
    model.fit(df_without_index)
    anomalies = pd.Series(model.predict(df_without_index)).apply(lambda x: True if (x == -1) else False)
    return anomalies

In [ ]:
config={
    'anomaly_column':'isolation_ad',
    'legend_name': 'collective anomaly',
    'color':'rgba(255,217,61,0.8)'
}

anomalies = isolation_forest(df)
df_anomalies = _join_df_with_anomaly(df_anomalies,anomalies,config['anomaly_column'])
plot_anomalies(df_anomalies,y_column,config)


## Plot two types of the anomalies

In [ ]:
def plot(df, y_column, configs):

    figure = go.Figure()

    # plot baseline
    figure.add_trace(go.Scatter(name=y_column, x = df.index, y=df[y_column], marker=dict(color='rgba(50,50,50,0.3)')))

    # plot both levelshift and collective anomalies

    for config in configs:
        anomaly_df = df
        for anomaly_type, status in config['conditions'].items():
            anomaly_df = anomaly_df[anomaly_df[anomaly_type]==status]

        figure.add_trace(go.Scatter(name=config['legend_name'], x = anomaly_df.index, y=anomaly_df[y_column],
            mode='markers',
            marker=dict(color=config['style']['color'],size=10)))

        figure.update_layout(
            title= 'DAU (simulated)',
            xaxis_title='date',
            yaxis_title='DAU',
            legend_title="Anomaly Type",
        )


    figure.show()
    figure.write_image("img/anomlay.png")

In [ ]:
!pip install -U kaleido

In [ ]:
import kaleido

In [ ]:
configs=[
    {
        'anomaly_column':'levelshift_ad',
        'legend_name':'Level shift warning',
        'conditions':{
            'levelshift_ad':True,
            'isolation_ad':False,
        },
        'style':{
            'color': 'rgba(249,123,34,0.8)',
            'marker_size': 10
        }
    },
    {
        'anomaly_column':'isolation_ad',
        'legend_name':'Collective warning',
        'conditions':{
            'levelshift_ad':False,
            'isolation_ad':True,
        },
        'style':{
            'color': 'rgba(255,217,61,0.8)',
            'marker_size': 10
        }
    },
    {
        'legend_name':'Overlap warning',
        'conditions':{
            'levelshift_ad':True,
            'isolation_ad':True,
        },
        'style':{
            'color': 'rgba(223,46,56,0.8)',
            'marker_size': 10
        }
    },
]

plot(df_anomalies,y_column,configs)

FileNotFoundError: [Errno 2] No such file or directory: 'img/anomlay.png'

## Backtest...

In [ ]:
import pandas as pd

# Load data from URL
url = 'https://raw.githubusercontent.com/amenti4k/webchatter/main/trialthriveportcodau.csv?token=GHSAT0AAAAAACQ5GWW4VE4JHPCCFCK5GAGSZQ7LZWQ'
filename = 'url'
x_column = 'Date'
y_column = 'DAU'
df = load_data(filename, x_column, y_column)

# Plot the baseline
plot_base_line(df, y_column)

# Define the date range for backtesting
dates = pd.date_range(start="2022-11-03", end="2022-11-17")

# Define the data configuration
df_config = {
    'filename': filename,
    'x_column': x_column,
    'y_column': y_column,
}

# Define the level shift anomaly detection configuration
level_shift_config = {
    'anomaly_column': 'levelshift_ad',
    'legend_name': 'levelshift anomaly',
    'color': 'rgba(249,123,34,0.8)'
}

# Define the isolation forest anomaly detection configuration
isolation_forest_config = {
    'anomaly_column': 'isolation_ad',
    'legend_name': 'collective anomaly',
    'color': 'rgba(255,217,61,0.8)'
}

# Define the plot configurations
fig_configs = [
    {
        'anomaly_column': 'levelshift_ad',
        'legend_name': 'Level shift warning',
        'conditions': {
            'levelshift_ad': True,
            'isolation_ad': False,
        },
        'style': {
            'color': 'rgba(249,123,34,0.8)',
            'marker_size': 10
        }
    },
    {
        'anomaly_column': 'isolation_ad',
        'legend_name': 'Collective warning',
        'conditions': {
            'levelshift_ad': False,
            'isolation_ad': True,
        },
        'style': {
            'color': 'rgba(255,217,61,0.8)',
            'marker_size': 10
        }
    },
    {
        'legend_name': 'Overlap warning',
        'conditions': {
            'levelshift_ad': True,
            'isolation_ad': True,
        },
        'style': {
            'color': 'rgba(223,46,56,0.8)',
            'marker_size': 10
        }
    },
]

# Define the modified backtest function
def backtest(dates, df_config, level_shift_config, isolation_forest_config, fig_configs):
    df = load_data(df_config['filename'], df_config['x_column'], df_config['y_column'])

    for end_date in dates:
        df_current = df[:end_date]

        anomalies = level_shift_anomaly(df_current)
        df_anomalies = _join_df_with_anomaly(df_current, anomalies, level_shift_config['anomaly_column'])

        anomalies = isolation_forest(df_current)
        df_anomalies = _join_df_with_anomaly(df_anomalies, anomalies, isolation_forest_config['anomaly_column'])

        plot(df_anomalies, df_config['y_column'], fig_configs)

# Run the backtest function with the defined configurations
backtest(dates, df_config, level_shift_config, isolation_forest_config, fig_configs)

<ipython-input-6-cf3ff2bf9f53>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-6-cf3ff2bf9f53>:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



FileNotFoundError: [Errno 2] No such file or directory: 'img/anomlay.png'